# Resampled PSFs from _JWST_/NIRCam Imaging

This example notebook shows how `spike` interacts with _JWST_/NIRCam imaging to generate a resampled PSF. This notebook uses observations the following files:
- jw02514162001_03201_00001_nrca2_cal.fits
- jw02514162001_03201_00002_nrca2_cal.fits
- jw02514162001_03201_00003_nrca2_cal.fits

The easiest way to find these data in the MAST archive is by searching for observation IDs jw02514162001_03201_00001_nrca2, jw02514162001_03201_00002_nrca2, jw02514162001_03201_00003_nrca2. `spike` uses calibrated, but not-yet co-added images (i.e., "Level 2" data products), so be sure that your download includes 'cal' files.

In [1]:
from spike.psf import jwst # import the relevant top-level module from spike

datapath = '' #'/path/to/nircam/data'


The outputs are very long due to all of the print statements from the tweak, PSF generation, and drizzling steps. Outputs have been cleared for cells where `spike.psf.jwst` was run.